In [2]:
! pip install -r requirements.txt
! pip install imageio[ffmpeg]

  Using cached scikit_image-0.21.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.9 MB)
  Using cached clearml-1.12.2-py2.py3-none-any.whl (1.1 MB)
  Using cached tifffile-2023.7.10-py3-none-any.whl (220 kB)
  Using cached imageio-2.31.1-py3-none-any.whl (313 kB)
  Using cached PyWavelets-1.4.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.9 MB)
  Using cached scikit_image-0.20.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
  Using cached lazy_loader-0.3-py3-none-any.whl (9.1 kB)
  Using cached scikit_image-0.19.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.0 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached PyJWT-2.4.0-py3-none-any.whl (18 kB)
  Using cached furl-2.1.3-py2.py3-none-any.whl (20 kB)
  Using cached pathlib2-2.3.7.post1-py2.py3-none-any.whl (18 kB)
  Using cached orderedmultidict-1.0.1-py2.py3-none-any.whl (11 kB)
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip

In [3]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# Laptop
%env CLEARML_API_ACCESS_KEY=67ZJZCBY9YK9I9LD4LU0
%env CLEARML_API_SECRET_KEY=xhnETxvNy1pqOp0XNDY9QsJi5aHC5gFj580yBENXw4SBLrz0S9

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=67ZJZCBY9YK9I9LD4LU0
env: CLEARML_API_SECRET_KEY=xhnETxvNy1pqOp0XNDY9QsJi5aHC5gFj580yBENXw4SBLrz0S9


In [4]:
! nvidia-smi

Sun Aug 13 04:22:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import os
import io
from clearml import Task

In [7]:
from keras.models import Model, Sequential,load_model
from keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
import keras
import tensorflow as tf

from sklearn.model_selection import KFold

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import curve_fit, minimize

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import exposure
from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML

import time

from tensorflow.python import debug as tf_debug

import models.modded_unet as modded_unet
import models.loss

import data_loader

from tqdm import tqdm

In [8]:
###################################
# TensorFlow wizardry
# config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
# config.gpu_options.allow_growth = True
 
# Create a session with the above options specified.
# keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [9]:
model_type = modded_unet

exp_name = "HoloUNet_cell_counting"

max_x = 512
max_y = 512

## Define Model

In [10]:
def build_unet(bb=None):
    unet = model_type.get_model(1,max_x,max_y,32)
    #unet.compile(optimizer='adadelta', loss=models.loss.npcc_fft)
    unet.compile(optimizer='adadelta', loss=models.loss.npcc_npcc_fft)

    return unet

In [12]:
build_unet().summary()

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

## Load Dataset

In [10]:
! ls

HoloUNet.ipynb	data		models		  weighted_models
__pycache__	data_loader.py	requirements.txt


In [15]:
#Load Data
dl = data_loader.data_loader(max_x,max_y)
train_data_x, train_data_y, test_data_x, test_data_y, train_input_filelist, test_input_filelist = dl.get_bj7_ds()

## Train

In [16]:
n_folds = 5
batch_size = 8

#keras.backend.set_session(tf.Session())

In [13]:
kf = KFold(n_splits=n_folds, shuffle=True)

i=0

task = Task.init(project_name='cell_counting', task_name=exp_name)

for train_ind, val_ind in kf.split(train_data_x,train_data_y):
    train_x = train_data_x[train_ind]
    train_y = train_data_y[train_ind]
    
    val_x = train_data_x[val_ind]
    val_y = train_data_y[val_ind]

    checkpoint = ModelCheckpoint("weighted_models/{}_{}.h5".format(exp_name,i), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    model = build_unet()
    model.fit(train_x,train_y,
          batch_size=batch_size,
          epochs=100,
          validation_data=(val_x,val_y),
          verbose=1,
          shuffle=True,
          callbacks=[checkpoint])
    
    task.upload_artifact(name='model', artifact_object="weighted_models/{}_{}.h5".format(exp_name,i))

    #K.clear_session()

    i+=1
    
    break

ClearML Task: created new task id=b1ea9b2243bc4204b5b0828ab99f050a
2023-07-30 09:22:27,273 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/3dd431f59174464291b55ada9b70de7e/experiments/b1ea9b2243bc4204b5b0828ab99f050a/output/log
Epoch 1/100
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2023-07-30 09:22:37.074 tensorflow-2-6-gpu--ml-g4dn-xlarge-c97c437bb7a567fed6fc3e40998a:2184 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-07-30 09:22:37.654 tensorflow-2-6-gpu--ml-g4dn-xlarge-c97c437bb7a567fed6fc3e40998a:2184 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabl

## Testing

In [17]:
#best_model = load_model('weighted_models/unet_fringe_contrast_fibroblast_fullfft_log_22-12-19--17-31_split1.h5.h5')
best_model = load_model('weighted_models/HoloUNet_cell_counting_0.h5', custom_objects={'npcc_npcc_fft': models.loss.npcc_npcc_fft})

In [13]:
for i in tqdm(np.arange(test_data_x.shape[0])):
    result = best_model.predict(test_data_x[i].reshape((1,max_x,max_y,1)))
    #result = model.predict(train_data_x[i].reshape((1,max_x,max_y,1)))
    #plt.imshow(train_data_y[0].reshape(max_x, max_y))
    plt.imshow(result.reshape(max_x, max_y))
    #print(test_data_x[20].reshape(max_x, max_y))
    #file_head = '.'.join(src_video_path.split('.')[:-1])
    filename = f'test.tiff'
    
    break
    #print(test_data_x[i].reshape((1,max_x,max_y,1)).shape)

NameError: name 'test_data_x' is not defined

## Cell Counting

In [10]:
src_video_paths = [
    #'data/OneDrive_1_23-07-2023/CAF/10 July/CAF_P16_100723.avi',
    #'data/OneDrive_1_23-07-2023/CAF/14 Jul/CAF P17.avi',
    #'data/OneDrive_1_23-07-2023/CAF/17 Jul/CAF P18.avi',
    #'data/OneDrive_1_23-07-2023/L929/11 July/L929_P9_1_T75_110723/L929_P9_recounted_110723.avi',
    #'data/OneDrive_1_23-07-2023/L929/11 July/L929_P9_2_T25_110723/L929_P9_T25_071123.avi',
    #'data/OneDrive_1_23-07-2023/L929/12 July/L929_P9_2_T25 _120723/L929_P9_2_T25 _120723.avi',
    #'data/OneDrive_1_23-07-2023/L929/12 July/L929_P10_1_T25_120723/L929_P10_1_T25_120723.avi'
    'data/lowlight/l929_1mW_ND0.6.avi',
    #'data/lowlight/l929_1mW.avi',
    #'data/lowlight/l929_5mW.avi',
]

In [11]:
import numpy as np
from PIL import Image
from scipy.optimize import curve_fit
import random

def fit_func(data,a,b,c,d,e,f):
    l_x = len(data[:,0])
    l_y = len(data[0])

    y = data[0]
    x = np.repeat(range(1,l_x+1), l_y)
    y = np.tile(range(1,len(data[0])+1), ((l_x,1))).flatten()

    '''print("X: ")
    print(x)
    print(x.shape)
    print("Y: ")
    print(y)
    print(y.shape)'''

    f = a+b*x+c*y+d*x*y+e*np.power(x,2)+f*np.power(y,2)
    return f

def curve_removal(pp, params=None):
    x,y = pp.shape
    lin_x = np.linspace(1,x,x)
    lin_y = np.linspace(1,y,y)
    tile_x = np.tile(lin_y,(x,1))
    tile_y = np.tile(lin_x.transpose(),(y,1)).transpose()

    if params is not None:
        popt = params
    else:
        popt = curve_params(pp)

    return popt[0]+ popt[1]*tile_x+popt[2]*tile_y+popt[3]*tile_x*tile_y+popt[4]*np.power(tile_x,2)+popt[5]*np.power(tile_y,2)

def curve_params(pp):
    x,y = pp.shape
    lin_x = np.linspace(1,x,x)
    lin_y = np.linspace(1,y,y)
    tile_x = np.tile(lin_y,(x,1))
    tile_y = np.tile(lin_x.transpose(),(y,1)).transpose()

    xy = np.tile(lin_x.flatten(),(y,1))
    z = pp.transpose().flatten()

    popt, pcov = curve_fit(fit_func,xy,z) # , bounds=([0,0,0,0,0,0],[np.inf,np.inf,np.inf,np.inf,np.inf,np.inf])

    '''print(tile_x)
    print(tile_y)
    print(tile_x.shape)
    print(tile_y.shape)
    print((tile_x*tile_y).shape)
    print(np.power(tile_y,2).shape)
    print("Popt")
    print(popt)
    print(pcov)
    print("PP shape")
    print(pp.shape)
    print("Z: ")
    print(z)'''

    return popt

In [12]:
class data_loader_video:
    def __init__(self,max_x,max_y):
        self.max_x = max_x
        self.max_y = max_y
    
    def get_data_large(self, folder,noisy_files,clean,duplicates,threshold=0.25,noisy=False,flipped=False, augment=False):    
        x_list = []
        y_list = []

        files = []

        noisy_imgs = []
        clean_imgs = []

        filelist = []

        with Image.open(f'{folder}{clean[0]}') as c_img:
            #if len(c_img.shape) == 3:
            #    c_img = c_img.convert('L')
            clean_img = np.asarray(c_img)

        if flipped:
            clean_img = np.flip(clean_img,axis=0)

        curve = curve_removal(clean_img,None)
        clean_min = np.min(clean_img-curve)
        curveless_clean = clean_img-curve-clean_min

        #curveless_clean = (curveless_clean-np.mean(curveless_clean))/np.std(curveless_clean)

        for a in range(len(noisy_files)):
            with Image.open(f'{folder}{noisy_files[a]}') as n_img:
                noisy_img = np.asarray(n_img) #.convert('L')

                if flipped:
                    noisy_img = np.flip(noisy_img,axis=0)

                curve = curve_removal(noisy_img,None)
                noisy_min = np.min(noisy_img-curve)
                curveless_noisy = noisy_img-curve-noisy_min

                #curveless_noisy = (curveless_noisy-np.mean(curveless_noisy))/np.std(curveless_noisy)

                noisy_imgs.append(curveless_noisy)

        for a in range(len(noisy_files)): 
            for i in range(duplicates):
                #clean_img = np.asarray(Image.open(f'{folder}{clean[0]}.tif'))
                noisy_img = noisy_imgs[a] #np.asarray(Image.open(f'{folder}{noisy[a]}.tif'))
                noisy_npcc = 0
                c = 0 

                while noisy_npcc < threshold and c < 1000:
                    #x_list.append(np.load(f'{self.noisy_folder}{file}').reshape((max_x,max_y,1)))
                    shifted_x = np.random.randint(clean_img.shape[0]-self.max_x)
                    shifted_y = np.random.randint(clean_img.shape[1]-self.max_y)

                    cropped_noisy = noisy_img[shifted_x:shifted_x+self.max_x,shifted_y:shifted_y+self.max_y]

                    if noisy:
                        noise = np.random.normal(scale=1,size=(self.max_x,self.max_y))
                        #noise = sp.misc.imresize(noise,(max_x,max_y))
                        cropped_noisy = cropped_noisy + noise

                    cropped_clean = curveless_clean[shifted_x:shifted_x+self.max_x,shifted_y:shifted_y+self.max_y]

                    noisy_npcc = abs(npcc(cropped_noisy,cropped_clean))
                    c += 1

                #curve = curve_removal(cropped_noisy,None)
                #noisy_min = np.min(cropped_noisy-curve)
                #curveless_noisy = cropped_noisy-curve-noisy_min

                cropped_noisy = (cropped_noisy-np.mean(cropped_noisy))/(np.std(cropped_noisy))
                cropped_clean = (cropped_clean-np.mean(cropped_clean))/(np.std(cropped_clean))

                if noisy_npcc >= threshold:
                    
                    
                #curve = curve_removal(cropped_clean,None)
                #clean_min = np.min(cropped_clean-curve)
                #curveless_clean = cropped_clean-curve-clean_min
                    if augment:
                        if np.random.randint(10) < 2:
                            cropped_noisy = np.flip(cropped_noisy,axis=0)
                            cropped_clean = np.flip(cropped_clean,axis=0)
                        
                        if np.random.randint(10) < 2:
                            cropped_noisy = np.flip(cropped_noisy,axis=1)
                            cropped_clean = np.flip(cropped_clean,axis=1)
                
                    #y_img_fft = np.fft.fftshift(np.fft.fft2(cropped_clean))
                    #y_img_bpass = y_img_fft.copy()
                    #y_img_bpass[200:300,200:300] = 0
                    #y_img_bpass_ifft = np.abs(np.fft.ifft2(np.fft.ifftshift(y_img_bpass)))
                
                    x_list.append(cropped_noisy.reshape((self.max_x,self.max_y,1)))
                    y_list.append(cropped_clean.reshape((self.max_x,self.max_y,1)))
                    #y_list.append(y_img_bpass_ifft.reshape((self.max_x,self.max_y,1)))
                    filelist.append(noisy_files[a])
                else:
                    print("Failed to find suitable image:",noisy_files[a])
                    break

        x_array = np.asarray(x_list)
        y_array = np.asarray(y_list)
                        
        return x_array, y_array, filelist

    def get_bj7_ds(self, num_imgs=100, threshold=0.1):
        folder = "data/observed/20x_0_45NA/19_11_19/"

        #train_input_filelist = ['NE20_1'] #'NE10NE20_2','NE01NE10NE20_2','NE02NE10NE20_2','NE03NE10NE20_2','NE04NE10NE20_2'
        #train_target_filelist = ['NE00_1']

        #test_input_filelist = ['NE20_5'] #,'NE10NE20_5','NE01NE10NE20_5','NE02NE10NE20_5','NE03NE10NE20_5','NE04NE10NE20_5'
        #test_target_filelist = ['NE00_5']

        train_input_filelist = []
        train_target_filelist = []

        test_input_filelist = []
        test_target_filelist = []

        train_data_x = None
        train_data_y = None

        test_data_x = None
        test_data_y = None

        #train_list = [20,1,2,4,6,7,8,9,11,13]
        #test_list = [18,16,15,14]
        
        train_list = [1,2,5,6,8,9,10]
        test_list = [14,15,18,20]
        
        
        for i in train_list:
            train_input_filelist.append(f'NE24_{i}')
            train_target_filelist.append(f'NE00_{i}')

            img_data_x,img_data_y, data_filelist = self.get_data_large(folder,[f'NE24_{i}.tif'],[f'NE00_{i}.tif'],100,0.02,False,augment=True)

            if len(img_data_x.shape) != 4:
                continue

            if train_data_x is None:
                train_data_x = img_data_x.copy()
                train_data_y = img_data_y.copy()
            else:
                train_data_x = np.append(train_data_x,img_data_x,axis=0)
                train_data_y = np.append(train_data_y,img_data_y,axis=0)

        for i in test_list:
            test_input_filelist.append(f'NE24_{i}')
            test_target_filelist.append(f'NE00_{i}')

            img_data_x, img_data_y, data_filelist = self.get_data_large(folder,[f'NE24_{i}.tif'],[f'NE00_{i}.tif'],100,0.02,False)

            if len(img_data_x.shape) != 4:
                continue

            if test_data_x is None:
                test_data_x = img_data_x.copy()
                test_data_y = img_data_y.copy()
            else:
                test_data_x = np.append(test_data_x,img_data_x,axis=0)
                test_data_y = np.append(test_data_y,img_data_y,axis=0)

        #train_data_x,train_data_y = get_data_large(['NE_30A_4','NE_30A_6'],['NE_20A_4','NE_30A_6'],800,False)
        #test_data_x, test_data_y = get_data_large(['NE_30A_5'],['NE_20A_5'],80,True)    

        return train_data_x, train_data_y, test_data_x, test_data_y, train_input_filelist, test_input_filelist

In [13]:
import cv2
from pathlib import Path
import imageio
import glob

In [14]:
def predict_frame(frame, model):
    #if len(frame.shape) == 3:
    #    frame = frame.convert('L')
    noisy_img = np.asarray(frame) #.convert('L')

    #print(f"{noisy_img.shape=}")

    #if out is None:
        #height, width, layers = noisy_img.shape
        #size = (width,height)
        #out = cv2.VideoWriter(filename=dst_path, fourcc=fourcc, fps=16, frameSize=size)

    #if flipped:
    #    noisy_img = np.flip(noisy_img,axis=0)

    curve = curve_removal(noisy_img,None)
    noisy_min = np.min(noisy_img-curve)
    curveless_noisy = noisy_img-curve-noisy_min
    curveless_noisy = (curveless_noisy-np.mean(curveless_noisy))/(np.std(curveless_noisy))

    output = np.zeros(noisy_img.shape)

    #curveless_noisy = (curveless_noisy-np.mean(curveless_noisy))/np.std(curveless_noisy)
    for x_shift in range(0, noisy_img.shape[0]-512, 512):
        for y_shift in range(0, noisy_img.shape[1]-512, 512):
            cropped_img = curveless_noisy[x_shift:x_shift+512, y_shift:y_shift+512]
            output[x_shift:x_shift+512, y_shift:y_shift+512] = best_model.predict(cropped_img.reshape((1,max_x,max_y,1))).reshape((max_x,max_y))

    for y_shift in range(0, noisy_img.shape[1]-512, 512):
        x_shift = noisy_img.shape[0]-512
        cropped_img = curveless_noisy[x_shift:x_shift+512, y_shift:y_shift+512]
        output[x_shift:x_shift+512, y_shift:y_shift+512] = best_model.predict(cropped_img.reshape((1,max_x,max_y,1))).reshape((max_x,max_y))

    for x_shift in range(0, noisy_img.shape[0]-512, 512):
        y_shift = noisy_img.shape[1]-512
        cropped_img = curveless_noisy[x_shift:x_shift+512, y_shift:y_shift+512]
        output[x_shift:x_shift+512, y_shift:y_shift+512] = best_model.predict(cropped_img.reshape((1,max_x,max_y,1))).reshape((max_x,max_y))

    x_shift = noisy_img.shape[0]-512
    y_shift = noisy_img.shape[1]-512
    cropped_img = curveless_noisy[x_shift:x_shift+512, y_shift:y_shift+512]
    output[x_shift:x_shift+512, y_shift:y_shift+512] = best_model.predict(cropped_img.reshape((1,max_x,max_y,1))).reshape((max_x,max_y))

    print(f"{output=}")
    #output += 1
    #output *= 127
    #output = output.astype(np.uint8)
    print(f"{np.max(output)=}")
    print(f"{np.min(output)=}")
    
    output -= np.min(output)
    output /= np.max(output)
    output *= 255
    output = output.astype(np.uint8)
    
    return output

def cell_count_cleanup(src_video_path, model):
    print(f"Cleaning video: {src_video_path}")
    src_cap = cv2.VideoCapture(src_video_path)
    #reader = imageio.get_reader(src_video_path)
    
    dst_path = Path('.'.join(src_video_path.split('.')[:-1]) + '_holo_unet.avi')
    print(f"Writing to {dst_path}")
    fourcc = cv2.VideoWriter_fourcc(*'XVID') 

    fps = src_cap.get(cv2.CAP_PROP_FPS)
    
    writer = imageio.get_writer(dst_path, fps=fps)
    
    out = None
    #pbar = tqdm(total=src_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(f"Frame count: {src_cap.get(cv2.CAP_PROP_FRAME_COUNT)}")
    
    i=0
    
    while(src_cap.isOpened()):
    #for i, frame in enumerate(reader):
        # Capture frame-by-frame
        #ret = True
        ret, frame = src_cap.read()
        
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            output = predict_frame(frame, model)
            
            #out.write(output)            
            writer.append_data(output)
        
            # Filename
            file_head = '.'.join(src_video_path.split('.')[:-1])
            filename = f'{file_head}_{i}.tif'

            # Using cv2.imwrite() method
            # Saving the image
            Image.fromarray(output).save(filename)
        
            i+=1
            #writer.append_data(frame)
            #pbar.update(1)
        else:
            print("Run out of frames?")
            break
    
    # After the loop release the cap object
    src_cap.release()
    
    # Destroy all the windows
    cv2.destroyAllWindows()        
    
def check_image_data(image_path, model):
    with Image.open(image_path) as c_img:
        output = predict_frame(c_img, model)
        folder = '/'.join(image_path.split('/')[:-1])
        filehead = image_path.split('/')[-1]
        file_head = '.'.join(filehead.split('.')[:-1])
        filename = f'data/predictions/19_11_19/{file_head}_holounet.tif'
        
        Image.fromarray(output).save(filename)
        #cv2.imwrite(filename, output)


In [18]:
for img_path in glob.glob('data/19_11_19/*'):
    print(f"{img_path=}")
    check_image_data(img_path, best_model)
    #break

img_path='data/19_11_19/NE24_16.tif'
output=array([[ 3.71631652e-01,  1.45645201e+00,  1.04995883e+00, ...,
         6.22338891e-01, -2.25643799e-01, -1.23254925e-01],
       [-4.23216760e-01,  1.05656815e+00,  2.08980727e+00, ...,
         2.08782840e+00, -5.44000138e-03, -6.36913359e-01],
       [-7.22861946e-01, -1.72162163e+00,  2.40961671e-01, ...,
         2.49488044e+00,  9.45387006e-01, -1.78284920e-03],
       ...,
       [ 1.24590623e+00,  2.17025137e+00,  3.88447672e-01, ...,
        -2.34745574e+00, -1.57236183e+00,  1.68474525e-01],
       [ 2.39637330e-01,  1.59436262e+00,  2.59439993e+00, ...,
         8.33809137e-01, -1.22390068e+00,  1.21420078e-01],
       [-3.22259724e-01, -3.57678622e-01,  8.31107914e-01, ...,
         1.05764496e+00,  2.12702021e-01, -2.81992167e-01]])
np.max(output)=23.135805130004883
np.min(output)=-10.787275314331055
img_path='data/19_11_19/NE24_7.tif'
output=array([[ 0.82147676,  0.27331689, -0.59842616, ..., -0.49467522,
        -0.66553968,  

In [19]:
for video_paths in src_video_paths:
    cell_count_cleanup(video_paths, best_model)

Cleaning video: data/lowlight/l929_1mW_ND0.6.avi
Writing to data/lowlight/l929_1mW_ND0.6_holo_unet.avi
output=array([[ 0.50799608,  0.49036738, -0.16482624, ..., -1.03326762,
        -0.66798371, -0.15337566],
       [ 0.5292179 ,  1.19891834,  0.19541763, ..., -1.55600226,
        -1.40842748, -0.43905818],
       [ 0.17092776,  1.67453933,  1.88040829, ..., -2.66510224,
        -2.23304868, -0.76702136],
       ...,
       [ 0.34847304,  2.44084835,  4.41002798, ...,  1.52785456,
         1.36856234,  0.88797146],
       [ 0.73260504,  2.02734947,  3.65987611, ...,  1.51606488,
         1.19028747,  0.91078949],
       [ 0.24913718,  0.2650882 ,  1.02689397, ...,  0.74797279,
         0.87657052,  0.75436568]])
np.max(output)=38.580101013183594
np.min(output)=-18.256940841674805
output=array([[ 0.57234991,  1.08155894,  1.13525474, ...,  0.07288627,
         0.28796679,  0.01072866],
       [ 0.46897808,  1.27161229,  1.45414782, ..., -0.51136225,
         0.09669319,  0.19259064],
 

In [26]:
# Load source video
# Create empty video
# For each frame, get prediction from ML
# Add outcome to new video

In [20]:
import shutil
shutil.make_archive('lowlight.zip', 'zip', 'data/lowlight')

'/root/Colab-Local/lowlight.zip.zip'